In [9]:
#stacked autoencoder based deep neural network for classification
import numpy as np
from scipy.io import loadmat

#Load data, shuffle and normalize
mat_contents = loadmat('data5.mat')
data = mat_contents['x']
np.random.shuffle(data)


def init_data():
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

X, y = init_data()

#Hold out
X_train, y_train = X[ :int(0.7 * len(X))], y[ :int(0.7 * len(X))]
X_val, y_val = X[ int(0.7 * len(X)): ], y[ int(0.7 * len(X)): ]

alpha = 0.5

#Sigmoid activation function
def sigmoid(x, derivative=False):
        if (derivative == True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

class NeuralNetwork(object):
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.W = {}
        self.a = {}
        self.b = {}
        
        #Initialize Weights
        for i in range(1, self.num_layers):
            self.W[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
            
        #Initialize biases
        for i in range(1, self.num_layers):
            self.b[i] = np.random.randn(self.sizes[i], 1)
        
        #Initialize activations
        for i in range(1, self.num_layers):
            self.a[i] = np.zeros([self.sizes[i], 1])
        
    #Forward pass to compute scores
    def forward_pass(self, X):
        
        self.a[0] = X
        
        for i in range(1, self.num_layers):
            self.a[i] = sigmoid(np.dot(self.W[i].T, self.a[i-1]) + self.b[i])

        return self.a[self.num_layers-1] 
    
    #Backward pass to update weights
    def backward_pass(self, X, Y, output):
        
        self.d = {}
        self.d_output = (Y - output) * sigmoid(output, derivative=True)
        self.d[self.num_layers-1] = self.d_output
        
        #Derivatives of the layers wrt loss
        for i in range(self.num_layers-1, 1, -1):
            self.d[i-1] = np.dot(self.W[i], self.d[i]) * sigmoid(self.a[i-1], derivative=True)
        
        #Updating weights
        for i in range(1, self.num_layers-1):
            self.W[i] += alpha * np.dot(self.a[i-1], self.d[i].T)
            
        #Updating biases
        for i in range(1, self.num_layers-1):
            self.b[i] += alpha * self.d[i]

    #Training helper function   
    def train(self, X, Y):
        X = np.reshape(X, (len(X), 1))
        output = self.forward_pass(X)
        self.backward_pass(X, Y, output)

    #Get weights    
    def get_W(self):
        return self.W
    
    #Load specified weights
    def load_W(self, W):
        self.W = W

    #Scores computation for given input    
    def get_a(self, x):
        x = np.reshape(x, (len(x), 1))
        self.forward_pass(x)
        return self.a
    
    #Helper function for autoencoder chaining
    def load_a(self, a):
        self.a = a

    

#cost function
def calc_cost(NN,x ,y):
    
    cost = 0
    for i in range(len(x)):
        x_ = np.reshape(x[i], (len(x[i]), 1))
        cost += 0.5 / len(x) * np.sum((y[i] - NN.forward_pass(x_)) ** 2)
    
    return cost

#Network initialization
autoencoder1 = NeuralNetwork([72, 60, 72])
autoencoder2 = NeuralNetwork([60,40,60])
autoencoder3 = NeuralNetwork([40, 30, 40])
NN = NeuralNetwork([72,60,40,30, 1])

#AE 1 pretraining
for i in range(200):
    for j, row in enumerate(X_train):
        row = np.reshape(row, (72,1))
        autoencoder1.train(row, row)
        
    cost = calc_cost(autoencoder1, X_train, X_train)
    if i%50 ==0:
      print("Epoch {}, cost {}".format(i, cost))
    
#Scores for AE 1
autoencoder2_input = []

for row in X_train:
    autoencoder2_input.append(autoencoder1.get_a(row)[1])

autoencoder2_input = np.array(autoencoder2_input)


#AE 2 pretraining
for i in range(200):
    for j, row in enumerate(autoencoder2_input):
        row = np.reshape(row, (60,1))
        autoencoder2.train(row, row)
        
    cost = calc_cost(autoencoder2, autoencoder2_input, autoencoder2_input)
    if i%50 ==0:
      print("Epoch {}, cost {}".format(i, cost))


#Scores for AE 2
autoencoder3_input = []

for row in autoencoder2_input:
    autoencoder3_input.append(autoencoder2.get_a(row)[1])

autoencoder3_input = np.array(autoencoder3_input)

#AE 3 pretraining
for i in range(200):
    for j, row in enumerate(autoencoder3_input):
        row = np.reshape(row, (40,1))
        autoencoder3.train(row, row)
        
    cost = calc_cost(autoencoder3, autoencoder3_input, autoencoder3_input)
    if i%50 ==0:
      print("Epoch {}, cost {}".format(i, cost))

#Final network weight initialization
W1 = autoencoder1.get_W()[1]
W2 = autoencoder2.get_W()[1]
W3 = autoencoder3.get_W()[1]
W_final = {}
W_final[1] = W1
W_final[2] = W2
W_final[3] = W3
W_final[4] = np.random.randn(30, 1)
NN.load_W(W_final)

#Training loop
for i in range(500):

    for j in range(len(X_train)):
        NN.train(X_train[j], y_train[j])

tp, tn, fp, fn = 0,0,0,0

for i in range(len(X_val)):

    x = np.reshape(X_val[i], (len(X_val[i]), 1))
    x = NN.forward_pass(x)
    p = 0 if x[0] < 0.5 else 1

    if p == 1 and y_val[i] == 1:
        tp += 1
    elif p == 0 and y_val[i] == 0:
        tn += 1
    elif p == 1 and y_val[i] == 0:
        fp += 1
    elif p == 0 and y_val[i] == 1:
        fn += 1

print(tp, fp)
print(fn, tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("accuracy = ", accuracy, "sensitivity = ", sensitivity, "specificity = ", specificity)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp


Epoch 0, cost 2985.0438692089747
Epoch 50, cost 2859.8079107687013
Epoch 100, cost 2863.3191282112566
Epoch 150, cost 2861.8041797859546
Epoch 0, cost 6.010090907942298
Epoch 50, cost 5.186879801320856
Epoch 100, cost 5.207901210815339
Epoch 150, cost 5.2068806593264565
Epoch 0, cost 2.811563101475409
Epoch 50, cost 2.5031379207394813
Epoch 100, cost 2.5025840233315195
Epoch 150, cost 2.497614544169333
282 25
28 310
accuracy =  0.9178294573643411 sensitivity =  0.9096774193548387 specificity =  0.9253731343283582
